In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
import pandas as pd
import re


In [2]:
def load_dataframe(file_path):
    try:
        df = pd.read_csv(file_path, sep='\t', error_bad_lines=False)
        print("Columns in the file:")
        for i, col in enumerate(df.columns):
            print(f"{i}. {col}")
        col_idx = int(input("Enter the index number of the column to use: "))
        selected_col = df.columns[col_idx]
        return df, selected_col
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None
    
def get_search_terms():
    choice = input("Would you like to (1) enter terms manually or (2) load terms from a file? Enter 1 or 2: ")
    if choice == '1':
        terms = input("Enter terms separated by commas: ").split(',')
        terms = [term.strip().lower() for term in terms]
    elif choice == '2':
        file_path = input("Enter the path to the terms file: ").replace('"', '')  # Remove any quotation marks
        with open(file_path, 'r') as file:
            content = file.read()
            terms = content.splitlines()
            terms = [term.strip().lower() for term in terms]
    else:
        print("Invalid input. Please enter 1 or 2.")
        return get_search_terms()
    return terms

def find_matches(df, col, terms):
    pattern = '|'.join([re.escape(term) for term in terms]) 
    df['Matched Terms'] = df[col].apply(lambda x: ', '.join(set(term for term in terms if re.search(re.escape(term), x.lower()))))
    matched_df = df[df['Matched Terms'] != '']
    return matched_df

def apply_exclusion_terms(df, col):
    exclusion_terms = input("Enter exclusion terms separated by commas (leave empty if none): ").split(',')
    if exclusion_terms == ['']:
        return df
    exclusion_pattern = '|'.join([re.escape(term.strip().lower()) for term in exclusion_terms])
    original_count = len(df)
    df = df[~df[col].str.lower().apply(lambda x: bool(re.search(exclusion_pattern, x)))]
    removed_count = original_count - len(df)
    print(f"Rows after exclusion: {len(df)}")
    print(f"Rows removed: {removed_count}")
    return df

def main():
    terms = get_search_terms()

    file1_path = input("Enter the path of the EMIS Medical Dictionary file: ").replace('"', '')  # Remove any quotation marks
    df1, col1 = load_dataframe(file1_path)
    
    if df1 is None:
        return

    matched_df = find_matches(df1, col1, terms)
    print(f"Initial matches found: {len(matched_df)}")
    if not matched_df.empty:
        # Display a snapshot of initial matches and ask for exclusion terms
        print(matched_df.head())  # Optionally display a few rows to help decide on exclusions
        matched_df = apply_exclusion_terms(matched_df, col1)

        output_directory = os.path.dirname(file1_path)
        default_filename = os.path.basename(file1_path).split('.')[0] + '_matched_results.csv'
        print(f"Files will be saved in the directory: {output_directory}")
        filename = input(f"Enter the filename for the saved results (default: {default_filename}): ").strip()
        if filename == '':
            filename = default_filename
        elif not filename.endswith('.csv'):
            filename += '.csv'  # Append .csv if not present
        output_file_path = os.path.join(output_directory, filename)
        
        try:
            matched_df.to_csv(output_file_path, index=False)
            print(f"Matched data saved to {output_file_path}")
        except PermissionError:
            print(f"Permission denied when trying to write to {output_file_path}. Please check your permissions.")
    else:
        print("No matches found.")

In [3]:
if __name__ == "__main__":
    main()

Would you like to (1) enter terms manually or (2) load terms from a file? Enter 1 or 2: 1
Enter terms separated by commas: hypertension, diabetes, colon
Enter the path of the EMIS Medical Dictionary file: "O:\CPRDTeam\CodeListPipeline\Sources\EMIS Medical Dictionary\202202_EMISMedicalDictionary_Alpha.txt"
Columns in the file:
0. Unnamed: 0
1. Med_Code_Id
2. Observations
3. OriginalReadCode
4. CleansedReadCode
5. Term
6. SNOMED_CT_Concept_ID
7. SnomedCTDescriptionId
8. Release
9. EmisCodeCategoryId
Enter the index number of the column to use: 5
Initial matches found: 1823
     Unnamed: 0 Med_Code_Id  Observations OriginalReadCode CleansedReadCode  \
82           82    a3135013        300000             G201          G201.00   
255         255    a8093016        400000            F4504          F450400   
319         319    a9986011           100          ESCTDR3              NaN   
351         351   a10928019            20         ESCTDI18              NaN   
386         386   a11743018